In [2]:
!pip install flask_cors

In [ ]:
############# Flask Modules Setup ##############
# !pip install flask_login 
# !pip install flask_cors
from flask import Flask, render_template, url_for, redirect, request, make_response, Response, jsonify
from flask_login import LoginManager, login_user, logout_user, login_required, UserMixin
from flask_cors import CORS  #no need in PA

#import MySQLdb
import pymysql.cursors

######### Initialize Flask App ##########

app = Flask(__name__)
CORS(app) #no need in PA
# app.config['SEND_FILE_MAX_AGE_DEFAULT']  #for PA

######### MySQL Database Setup ##########
#mark out this entire section, open and close the connection inside the album & API function
connection = pymysql.connect(host="localhost", 
#connection = MySQLdb.connect(host="localhost",
                     user="root",   
                     passwd="root", 
                     db="Lecture5_ABC",   
                     charset='utf8mb4')       

######### Login Manager Setup ##########

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
app.config['SECRET_KEY'] = "lkkajdghdadkglajkgah"  # secret_key?


@login_manager.user_loader
def load_user(user_id):
    return User(user_id)

class User(UserMixin):
  def __init__(self,id):
    self.id = id

############ Web Page Routes Setup ###############

@app.route("/")
@login_required
def home():
    return render_template('home.html')

@app.route("/login")
def login():
    message = 'Please login in first.'
    return render_template('login.html', message=message)

@app.route('/query_auth')   #query-string method
def query_auth():
    username = request.args.get("username")
    password = request.args.get("password")
    if username == "joyce@link.cuhk.edu.hk" and password == "joyce@cuhk":
        login_user(User(1))
        return make_response(message, 200, )
    else:
        message = 'Wrong password!'
        return make_response( messagte,401,{'WWW-Authenticate':'Basic realm="Login Required"'})

@app.route("/form_auth",methods=['POST'])  #form method
def form_auth():
    username = request.form['email']
    password = request.form['pwd']
    if username == "joyce@link.cuhk.edu.hk" and password == "joyce@cuhk":
        login_user(User(1))
        message = "Dear " + username + ", welcome to Naples album. Your login has been granted."
        return render_template('home.html', message=message)
    else:
        message = 'Wrong password!'
        return render_template('login.html',message=message)

@app.route('/album')
@login_required
def album():
    city = {"name":"Naples"}
    user = {"name":"Joyce"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM Naples"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:  #order them as same as the that in phpmyadmin
        dict['DestinationEn'] = i[0]
        dict['lat'] = i[1]
        dict['lng'] = i[2]
        dict['DesPageLink'] = i[3]
        dict['picUrl'] = i[4]
        dataset.append(dict.copy()) #markers.append(fld.copy()), append the copy, not the original dict
    
    return render_template('album.html',album_user=user, city=city, dataset=dataset)

@app.route('/logout/')
@login_required
def logout():
    logout_user()
    message = 'Thanks for logging out.'
    return render_template('login.html',message=message)

@app.errorhandler(500)
def internal_error(error):
    message = 'Wrong password!'
    return render_template('login.html',message=message),500

######### API Endpoints ##########    

@app.route('/api_auth', methods=['POST'])
def api_auth():
    username = request.json['username']
    password = request.json['password']
    if username == "joyce@link.cuhk.edu.hk" and password == "joyce@cuhk":
        login_user(User(1))
        message = "Dear " + username + ", welcome to Joyce's pages. Your login has been granted."
        return jsonify({'response':'ok!','message':message})
    else:
        message = 'Wrong password!'
        return jsonify({'response':'Invalid!','message':'Cannot authenticate.'})
    
@app.route('/api_album', methods=['POST'])
@login_required
def api_album():
    city = {"name":"Naples"}
    user = {"name":"Joyce"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM Naples"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:  #order them as same as the that in phpmyadmin
        dict['DestinationEn'] = i[0]
        dict['lat'] = i[1]
        dict['lng'] = i[2]
        dict['DesPageLink'] = i[3]
        dict['picUrl'] = i[4]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    return jsonify({'Album': dataset})

######### Run Flask Web App at Port 9030 ##########    

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9050, app)  #try to use a new number when console says "DevTools failed to parse SourceMap" 
    
# if __name__ == '__main__':   # pythonanywhere
   # app.run(debug = True)
    
    
    

 * Running on http://localhost:9050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Mar/2020 00:23:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2020 00:23:04] "GET /album HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2020 00:23:19] "GET /logout/ HTTP/1.1" 200 -
[2020-03-05 00:24:38,334] ERROR in app: Exception on /query_auth [GET]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.7/site-packages/flask_cors/extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/anaconda3/lib/python3.7/site-packages/fl